* [#49](https://github.com/salgo60/SCB-Wikidata/issues/49)
* [Notebook Regeringskansliet_49.ipynb](https://github.com/salgo60/SCB-Wikidata/blob/main/notebook/Regeringskansliet_49.ipynb)
* result links_regeringen
   * [2026_01_07.html](http://salgo60.github.io/SCB-Wikidata/notebook/resultsRegeringen/links_regeringen_v1_2026_01_07.html)
   * [2026_01_08.html](http://salgo60.github.io/SCB-Wikidata/notebook/resultsRegeringen/links_regeringen_v1_2026_01_08.html)
 

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2026-01-08 22:09:37


In [2]:

SCRIPT_NAME = "Regeringskansliet_49.ipynb"
SCRIPT_URL = (
    "https://github.com/salgo60/SCB-Wikidata/"
    "blob/master/notebook/Regeringskansliet_49.ipynb"
) 


In [3]:
def read_domains(file_path):
    print(f"[DEBUG] Reading domains from: {file_path}")
    df = pd.read_csv(file_path, header=0)   # <- skip header row
    domains_list = df.iloc[:, 0].dropna().unique().tolist()
    print(f"[DEBUG] Found {len(domains_list)} domains.")
    return domains_list


In [4]:
import requests

def fetch_sitematrix_df():
    url = "https://meta.wikimedia.org/w/api.php"
    params = {
        "action": "sitematrix",
        "format": "json"
    }
    headers = {
        "User-Agent": "salgo60-language-fetcher/1.0 (salgo60@msn.com)"
    }

    print("[DEBUG] Fetching sitematrix…")
    r = requests.get(url, params=params, headers=headers)
    r.raise_for_status()

    if "application/json" not in r.headers.get("Content-Type", ""):
        raise ValueError("Server returned non-JSON response")

    data = r.json()["sitematrix"]

    rows = []

    # --- language-specific sites ---
    for key, lang_block in data.items():
        if not key.isdigit():
            continue  # skip "count", "specials"

        lang_code = lang_block.get("code")
        lang_name = lang_block.get("name")

        for site in lang_block.get("site", []):
            rows.append({
                "lang_code": lang_code,
                "lang_name": lang_name,
                "project": site.get("project"),
                "url": site.get("url"),
                "dbname": site.get("dbname"),
                "site_name": site.get("sitename"),
                "closed": site.get("closed", False)
            })

    # --- special wikis (Wikidata, Commons, Meta, etc.) ---
    for site in data.get("specials", []):
        rows.append({
            "lang_code": "special",
            "lang_name": "special",
            "project": site.get("project"),
            "url": site.get("url"),
            "dbname": site.get("dbname"),
            "site_name": site.get("sitename"),
            "closed": site.get("closed", False)
        })

    return pd.DataFrame(rows)


In [5]:
import requests
import pandas as pd


HEADERS = {
    "User-Agent": "salgo60-language-fetcher/2.0 (https://github.com/salgo60)"
}


df_lang_fetch = fetch_sitematrix_df()
df_lang_fetch["closed"] = df_lang_fetch["closed"].fillna(False).astype(bool)

df_lang_wikipedia = df_lang_filtered = df_lang_fetch[
    (df_lang_fetch["site_name"] == "Wikipedia") &
    (df_lang_fetch["lang_name"].str.lower() != "special")]

#df_lang_wikipedia.to_csv("test.csv")
df_lang_wikipedia.info()

[DEBUG] Fetching sitematrix…
<class 'pandas.core.frame.DataFrame'>
Index: 184 entries, 0 to 937
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lang_code  184 non-null    object
 1   lang_name  183 non-null    object
 2   project    0 non-null      object
 3   url        184 non-null    object
 4   dbname     184 non-null    object
 5   site_name  184 non-null    object
 6   closed     184 non-null    bool  
dtypes: bool(1), object(6)
memory usage: 10.2+ KB


In [6]:
import os

# Get the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)



Current Working Directory: /Users/salgo/Documents/GitHub/SCB-Wikidata/notebook


In [7]:
import os
import time
import random
import requests
import pandas as pd
from urllib.parse import urlparse
from tqdm.notebook import tqdm
file_path_domain = "sources/domains_Regeringskansliet.csv"
domains = read_domains(file_path_domain)
print(domains)


[DEBUG] Reading domains from: sources/domains_Regeringskansliet.csv
[DEBUG] Found 1 domains.
['regeringen.se']


In [8]:
# -----------------------------------------------------------
# Fetch exturlusage entries for one lang/domain
# -----------------------------------------------------------
def fetch_exturlusage(lang, domain):
    base = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "list": "exturlusage",
        "euquery": domain,
        "eulimit": "max"
    }
    while True:
        r = session.get(base, params=params, timeout=10)
        try:
            data = r.json()
        except ValueError:
            print(f"[WARN] {lang}: JSON decode failed")
            break

        for item in data.get("query", {}).get("exturlusage", []):
            yield {
                "lang": lang,
                "page_title": item.get("title"),
                "url": item.get("url"),
                "wiki_link": f"https://{lang}.wikipedia.org/wiki/{item.get('title').replace(' ', '_')}"
            }

        if "continue" not in data:
            break
        params.update(data["continue"])
        time.sleep(0.3)

In [9]:

# -------------------------
# Session & helpers
# -------------------------
session = requests.Session()
session.headers.update({"User-Agent": "SCB-LinkAudit/1.0 (https://www.scb.se/)"})

# we need some filtering  

print("Antal Språk:",len(df_lang_wikipedia ))
results = []
for _, row in df_lang_wikipedia.iterrows():
    lang = row["lang_code"]
    url  = row["url"]
    lang_name = row["lang_name"]
    before = len(results)
    #print(lang, url, lang_name,domains)
    for entry in fetch_exturlusage(lang, domains):
        results.append(entry)     
    after = len(results) 
    links = after-before
    print(lang, url, lang_name," - ", links)
    


Antal Språk: 184
aa https://aa.wikipedia.org Qafár af  -  0
ace https://ace.wikipedia.org Acèh  -  0
af https://af.wikipedia.org Afrikaans  -  3
ak https://ak.wikipedia.org None  -  0
ami https://ami.wikipedia.org Pangcah  -  0
an https://an.wikipedia.org aragonés  -  0
ast https://ast.wikipedia.org asturianu  -  6
av https://av.wikipedia.org авар  -  0
avk https://avk.wikipedia.org Kotava  -  0
ay https://ay.wikipedia.org Aymar aru  -  0
bar https://bar.wikipedia.org Boarisch  -  1
bbc https://bbc.wikipedia.org Batak Toba  -  0
bcl https://bcl.wikipedia.org Bikol Central  -  4
bi https://bi.wikipedia.org Bislama  -  0
bm https://bm.wikipedia.org bamanankan  -  0
bo https://bo.wikipedia.org བོད་ཡིག  -  0
br https://br.wikipedia.org brezhoneg  -  0
bs https://bs.wikipedia.org bosanski  -  1
btm https://btm.wikipedia.org Batak Mandailing  -  0
bug https://bug.wikipedia.org Basa Ugi  -  0
bxr https://bxr.wikipedia.org буряад  -  0
cbk-zam https://cbk-zam.wikipedia.org Chavacano de Zamboan

In [10]:
df_regeringen = pd.DataFrame(results)
df_regeringen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9626 entries, 0 to 9625
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lang        9626 non-null   object
 1   page_title  9626 non-null   object
 2   url         9626 non-null   object
 3   wiki_link   9626 non-null   object
dtypes: object(4)
memory usage: 300.9+ KB


In [11]:
import pandas as pd

# --- Stats ---
total_links = len(df_regeringen)
total_unique_links = df_regeringen['url'].nunique()
num_languages = df_regeringen['lang'].nunique()
langs_sorted = df_regeringen['lang'].value_counts()

print("Total links:", total_links)
print("Total unique links:", total_unique_links)
print("Number of languages:", num_languages)
print("\nLanguages with most links:")
print(langs_sorted.to_string())


Total links: 9626
Total unique links: 5594
Number of languages: 36

Languages with most links:
lang
sv        6351
en        1971
no         387
fi         180
de         149
es         127
id          66
ro          59
zh          50
da          45
it          42
ja          37
pl          36
vi          21
ms          17
nn          11
nl          11
ig           8
sq           7
fo           7
simple       7
ha           6
ast          6
gl           4
is           4
bcl          4
af           3
sw           2
pcm          1
sco          1
bs           1
min          1
bar          1
tt           1
eu           1
ff           1


In [12]:
# --- Stats ---
total_links = len(df_regeringen)
total_unique_links = df_regeringen["url"].nunique()
langs_with_hits = sorted(df_regeringen["lang"].unique())

num_languages_found = len(langs_with_hits)
num_languages_checked = len(df_lang_wikipedia)        # alla språk som genomsöktes
num_languages_found = df_regeringen['lang'].nunique()


In [13]:
num_languages_checked 

184

In [14]:
import requests
from requests.exceptions import RequestException

SOFT_404_PHRASES = [
    "Sidan kan inte hittas",
    "sidan tagits bort",
    "felaktig adress",
    "kontakta registrator",
    "Regeringskansliets arkiv",
]

ROOT_CANONICAL = "https://www.regeringen.se"


def norm(u):
    return u.rstrip("/").lower()


def check_regeringen_url(url, timeout=15):
    try:
        r = requests.get(
            url,
            allow_redirects=True,
            timeout=timeout,
            headers={
                "User-Agent": "LinkChecker/1.0 (research; contact info)"
            }
        )
    except RequestException as e:
        return {
            "url": url,
            "status": "error",
            "reason": str(e),
        }

    final_url = r.url
    status_code = r.status_code
    text = r.text or ""

    # 1. Hard HTTP error
    if status_code >= 400:
        return {
            "url": url,
            "status": "dead",
            "reason": f"HTTP {status_code}",
            "final_url": final_url,
        }

    # 2. Redirected to site root (content missing)
    if norm(final_url) == norm(ROOT_CANONICAL) and norm(url) != norm(final_url):
        return {
            "url": url,
            "status": "dead",
            "reason": "redirect_to_root",
            "final_url": final_url,
        }

    # 3. Soft 404 detection
    lowered = text.lower()
    for phrase in SOFT_404_PHRASES:
        if phrase.lower() in lowered:
            return {
                "url": url,
                "status": "dead",
                "reason": "soft_404",
                "final_url": final_url,
            }

    # 4. Otherwise OK
    return {
        "url": url,
        "status": "ok",
        "final_url": final_url,
    }


In [15]:
def check_internet_archive(url, timeout=10):
    api = "https://archive.org/wayback/available"
    try:
        r = requests.get(
            api,
            params={"url": url},
            timeout=timeout,
            headers={"User-Agent": "LinkChecker/1.0"}
        )
        data = r.json()
    except Exception:
        return None

    snap = data.get("archived_snapshots", {}).get("closest")
    if snap and snap.get("available"):
        return snap.get("url")

    return None


In [16]:
df_regeringen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9626 entries, 0 to 9625
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lang        9626 non-null   object
 1   page_title  9626 non-null   object
 2   url         9626 non-null   object
 3   wiki_link   9626 non-null   object
dtypes: object(4)
memory usage: 300.9+ KB


In [17]:
from tqdm import tqdm  
import time
results = []
checked = set() 
urls = df_regeringen["url"].dropna().astype(str).unique()
for url in tqdm(
    urls,
    total=len(urls),
    desc="Checking regeringen.se links",
    unit="link",
):
    #print("url:", url)
    if url in checked:
        continue
    result = check_regeringen_url(url)  
    ia_url = check_internet_archive(url)
    result["ia_url"] = ia_url
    result["ia_status"] = "available" if ia_url else "missing"
    results.append(result)
    checked.add(url)
    time.sleep(0.3)  


Checking regeringen.se links: 100%|█████| 5594/5594 [6:53:19<00:00,  4.43s/link]


In [18]:
# HARD RESET
df_results = None

print("len(results):", len(results))

df_results = pd.DataFrame(results)

print("df_results.shape:", df_results.shape)
print(df_results.head(2))
print(df_results.tail(2))


len(results): 5594
df_results.shape: (5594, 6)
                                         url status    reason  \
0   http://regeringen.se/sb/d/14087/a/165866   dead  HTTP 404   
1  http://www.regeringen.se/sb/d/108/a/85634   dead  HTTP 404   

                                    final_url  \
0   https://regeringen.se/sb/d/14087/a/165866   
1  https://www.regeringen.se/sb/d/108/a/85634   

                                              ia_url  ia_status  
0  http://web.archive.org/web/20140111230654/http...  available  
1  http://web.archive.org/web/20140303053546/http...  available  
                                                    url status    reason  \
5592  https://www.regeringen.se/pressmeddelanden/202...     ok       NaN   
5593  https://www.regeringen.se/pressmeddelanden/202...   dead  HTTP 404   

                                              final_url  \
5592  https://www.regeringen.se/pressmeddelanden/202...   
5593  https://www.regeringen.se/pressmeddelanden/202...   

    

In [19]:
print("len(results):", len(results))
print("first:", results[0])
print("last:", results[-1])

len(results): 5594
first: {'url': 'http://regeringen.se/sb/d/14087/a/165866', 'status': 'dead', 'reason': 'HTTP 404', 'final_url': 'https://regeringen.se/sb/d/14087/a/165866', 'ia_url': 'http://web.archive.org/web/20140111230654/http://www.regeringen.se/sb/d/14087/a/165866', 'ia_status': 'available'}
last: {'url': 'https://www.regeringen.se/pressmeddelanden/2025/02/statsministern-och-justitieministern-bjuder-in-till-presstr%C3%A4ff/', 'status': 'dead', 'reason': 'HTTP 404', 'final_url': 'https://www.regeringen.se/pressmeddelanden/2025/02/statsministern-och-justitieministern-bjuder-in-till-presstr%C3%A4ff/', 'ia_url': None, 'ia_status': 'missing'}


In [20]:
df_results["status"].value_counts()

status
ok       3281
dead     2297
error      16
Name: count, dtype: int64

In [21]:
(df_results["status"] == "dead").mean()

0.4106185198426886

In [22]:
df_results[
    (df_results["status"] == "dead") &
    (df_results["ia_status"] != "available")
].shape[0]

532

In [23]:
type(results[0])

dict

In [24]:
results[:3]

[{'url': 'http://regeringen.se/sb/d/14087/a/165866',
  'status': 'dead',
  'reason': 'HTTP 404',
  'final_url': 'https://regeringen.se/sb/d/14087/a/165866',
  'ia_url': 'http://web.archive.org/web/20140111230654/http://www.regeringen.se/sb/d/14087/a/165866',
  'ia_status': 'available'},
 {'url': 'http://www.regeringen.se/sb/d/108/a/85634',
  'status': 'dead',
  'reason': 'HTTP 404',
  'final_url': 'https://www.regeringen.se/sb/d/108/a/85634',
  'ia_url': 'http://web.archive.org/web/20140303053546/http://www.regeringen.se/sb/d/108/a/85634',
  'ia_status': 'available'},
 {'url': 'https://www.regeringen.se/myndigheter-med-flera/nordiska-afrikainstitutet/',
  'status': 'ok',
  'final_url': 'https://www.regeringen.se/myndigheter-med-flera/nordiska-afrikainstitutet/',
  'ia_url': 'http://web.archive.org/web/20250427171446/https://www.regeringen.se/myndigheter-med-flera/nordiska-afrikainstitutet/',
  'ia_status': 'available'}]

In [25]:
results[0] is results[1]


False

In [26]:
df_regeringen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9626 entries, 0 to 9625
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lang        9626 non-null   object
 1   page_title  9626 non-null   object
 2   url         9626 non-null   object
 3   wiki_link   9626 non-null   object
dtypes: object(4)
memory usage: 300.9+ KB


In [27]:
#df_results = pd.DataFrame(results)  
df_results = (
    pd.DataFrame(results)
    .drop_duplicates(subset="url", keep="last")
)
df_regeringen = df_regeringen.merge(
    df_results,
    on="url",
    how="left",
) 

In [28]:
df_regeringen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9626 entries, 0 to 9625
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lang        9626 non-null   object
 1   page_title  9626 non-null   object
 2   url         9626 non-null   object
 3   wiki_link   9626 non-null   object
 4   status      9626 non-null   object
 5   reason      4291 non-null   object
 6   final_url   9601 non-null   object
 7   ia_url      8159 non-null   object
 8   ia_status   9626 non-null   object
dtypes: object(9)
memory usage: 677.0+ KB


In [29]:
len(results)

5594

In [30]:
status_counts = df_regeringen["status"].value_counts()

num_ok = status_counts.get("ok", 0)
num_dead = status_counts.get("dead", 0)
num_error = status_counts.get("error", 0)
num_total = len(status_counts)
print( "Ok ",num_ok) 
print( "Dead ",num_dead)
print( "Error ",num_error ) 
print( "Total ",num_total )

Ok  5335
Dead  4266
Error  25
Total  3


In [31]:
df_regeringen["status"].value_counts()

status
ok       5335
dead     4266
error      25
Name: count, dtype: int64

In [32]:
df_regeringen["reason"].value_counts()

reason
HTTP 404                                                                                                                                                                                                                                                                                                                                                                           4087
soft_404                                                                                                                                                                                                                                                                                                                                                                            130
redirect_to_root                                                                                                                                                                                                                                 

In [33]:
#df_regeringen["ia_url"].value_counts() 

In [34]:

df_regeringen["ia_status"].value_counts()

ia_status
available    8159
missing      1467
Name: count, dtype: int64

In [35]:
from datetime import date
import os

# Sätt datum
today = date.today().strftime("%Y_%m_%d")

# Se till att katalogen finns
os.makedirs("resultsRegeringen", exist_ok=True)

# Bygg filnamn
outfile = f"resultsRegeringen/links_Regeringen_v1_{today}.csv"

# Exportera
df_regeringen.to_csv(outfile, index=False, encoding="utf-8")

print(f"[OK] Exported {len(df_regeringen)} rows to {outfile}")


[OK] Exported 9626 rows to resultsRegeringen/links_Regeringen_v1_2026_01_09.csv


In [36]:
# fixa http://www.regeringen.se/ felet som inte ör ett fel 
#df_regeringen[["url", "final_url", "status", "reason"]] 
df_regeringen[
    df_regeringen["url"].isin(
        ["http://www.regeringen.se/", "https://www.regeringen.se/"]
    )
]

,lang,page_title,url,wiki_link,status,reason,final_url,ia_url,ia_status
27,da,Sveriges regeringer,http://www.regeringen.se/,https://da.wikipedia.org/wiki/Sveriges_regeringer,dead,redirect_to_root,https://www.regeringen.se/,None,missing
28,da,Sveriges statsministre,http://www.regeringen.se/,https://da.wikipedia.org/wiki/Sveriges_statsmi...,dead,redirect_to_root,https://www.regeringen.se/,None,missing
34,da,Sveriges statsminister,http://www.regeringen.se/,https://da.wikipedia.org/wiki/Sveriges_statsmi...,dead,redirect_to_root,https://www.regeringen.se/,None,missing
78,de,Regierung Reinfeldt,http://www.regeringen.se/,https://de.wikipedia.org/wiki/Regierung_Reinfeldt,dead,redirect_to_root,https://www.regeringen.se/,None,missing
135,de,Regierung Löfven I,https://www.regeringen.se/,https://de.wikipedia.org/wiki/Regierung_Löfven_I,ok,NaN,https://www.regeringen.se/,http://web.archive.org/web/20260108082306/http...,available
139,de,Bildungssystem in Schweden,https://www.regeringen.se/,https://de.wikipedia.org/wiki/Bildungssystem_i...,ok,NaN,https://www.regeringen.se/,http://web.archive.org/web/20260108082306/http...,available
202,de,Schwedische Regierung,https://www.regeringen.se/,https://de.wikipedia.org/wiki/Schwedische_Regi...,ok,NaN,https://www.regeringen.se/,http://web.archive.org/web/20260108082306/http...,available
337,en,User talk:Aprock/Archive 1,http://www.regeringen.se/,https://en.wikipedia.org/wiki/User_talk:Aprock...,dead,redirect_to_root,https://www.regeringen.se/,None,missing
370,en,Swedish nuclear weapons program,http://www.regeringen.se/,https://en.wikipedia.org/wiki/Swedish_nuclear_...,dead,redirect_to_root,https://www.regeringen.se/,None,missing
505,en,Håkan Sandberg,http://www.regeringen.se/,https://en.wikipedia.org/wiki/Håkan_Sandberg,dead,redirect_to_root,https://www.regeringen.se/,None,missing


In [37]:
ROOTS = {
    "http://www.regeringen.se/",
    "https://www.regeringen.se/",
}

mask_root_ok = (
    df_regeringen["url"].isin(ROOTS)
    & df_regeringen["final_url"].isin(ROOTS)
    & (df_regeringen["status"] == "dead")
)

df_regeringen.loc[mask_root_ok, "status"] = "ok"
df_regeringen.loc[mask_root_ok, "reason"] = "canonical root redirect"


In [38]:
num_total = len(df_regeringen)

num_dead = df_regeringen[df_regeringen["status"] == "dead"].shape[0]
num_ok = df_regeringen[df_regeringen["status"] == "ok"].shape[0]
num_error = df_regeringen[df_regeringen["status"] == "error"].shape[0]

print(num_total,num_dead,num_ok,num_error )

9626 4222 5379 25


In [39]:
pct_dead = round(100 * num_dead / num_total, 1)
pct_ok = round(100 * num_ok / num_total, 1)
pct_error = round(100 * num_error / num_total, 1)
print (pct_dead,pct_ok,pct_error)

43.9 55.9 0.3


In [42]:
#pct_archived = round(100 * num_dead_archived / num_dead, 1) if num_dead else 0
#pct_lost = round(100 * num_dead_lost / num_dead, 1) if num_dead else 0
#broken_stats = {
#    "total_links": num_total,
#    "broken_links": num_dead,
#    "broken_pct": pct_dead,
#    "broken_archived": num_dead_archived,
#    "broken_archived_pct": pct_archived,
#    "broken_lost": num_dead_lost,
#    "broken_lost_pct": pct_lost,
#}


In [43]:
lang_stats = (
    df_regeringen
    .groupby("lang")
    .agg(
        total_links=("url", "count"),
        broken_links=("status", lambda s: (s == "dead").sum()),
        archived_links=("ia_status", lambda s: (s == "available").sum()),
    )
    .reset_index()
)

lang_stats["broken_pct"] = (
    100 * lang_stats["broken_links"] / lang_stats["total_links"]
).round(1)

lang_stats["broken_lost"] = (
    lang_stats["broken_links"] - lang_stats["archived_links"])

top10_langs = (
    lang_stats
    .sort_values("total_links", ascending=False)
    .head(10)
)
top10_langs[
    [
        "lang",
        "total_links",
        "broken_links",
        "broken_pct",
        "archived_links",
        "broken_lost",
    ]
]

,lang,total_links,broken_links,broken_pct,archived_links,broken_lost
31,sv,6351,3011,47.4,5311,-2300
7,en,1971,622,31.6,1745,-1123
24,no,387,141,36.4,336,-195
11,fi,180,91,50.6,147,-56
6,de,149,68,45.6,123,-55
8,es,127,76,59.8,108,-32
15,id,66,33,50.0,59,-26
27,ro,59,14,23.7,56,-42
35,zh,50,32,64.0,38,-6
5,da,45,25,55.6,38,-13


In [44]:
assert (lang_stats["broken_links"] <= lang_stats["total_links"]).all()
assert (lang_stats["broken_pct"] <= 100).all()


In [45]:
from urllib.parse import urlparse

df = df_regeringen.copy()

df["domain"] = df["url"].apply(
    lambda u: urlparse(u).netloc.lower() if pd.notna(u) else None
)
domain_stats = (
    df
    .groupby("domain")
    .agg(
        total_links=("url", "count"),
        broken_links=("status", lambda s: (s == "dead").sum()),
        error_links=("status", lambda s: (s == "error").sum()),
    )
    .reset_index()
)
domain_stats["broken_pct"] = (
    100 * domain_stats["broken_links"] / domain_stats["total_links"]
).round(1)

domain_stats["error_pct"] = (
    100 * domain_stats["error_links"] / domain_stats["total_links"]
).round(1)


In [46]:
domain_stats 

,domain,total_links,broken_links,error_links,broken_pct,error_pct
0,justitie.regeringen.se,2,0,2,0.0,100.0
1,kultur.regeringen.se,2,0,2,0.0,100.0
2,regeringen.se,427,195,0,45.7,0.0
3,social.regeringen.se,2,0,2,0.0,100.0
4,statsradsberedningen.regeringen.se,4,0,4,0.0,100.0
5,utbildning.regeringen.se,1,0,1,0.0,100.0
6,www.justitie.regeringen.se,3,0,3,0.0,100.0
7,www.kultur.regeringen.se,1,0,1,0.0,100.0
8,www.regeringen.se,9175,4026,2,43.9,0.0
9,www.regeringen.se:80,1,1,0,100.0,0.0


In [47]:
status_counts = df["status"].value_counts()

num_ok = int(status_counts.get("ok", 0))
num_dead = int(status_counts.get("dead", 0))
num_error = int(status_counts.get("error", 0))
num_total = len(df)

pct_ok = round(100 * num_ok / num_total, 1)
pct_dead = round(100 * num_dead / num_total, 1)
pct_error = round(100 * num_error / num_total, 1)

# Broken links: archived vs lost
num_dead_archived = df[
    (df["status"] == "dead") & (df["ia_status"] == "available")
].shape[0]

num_dead_lost = num_dead - num_dead_archived


In [48]:
top_domains = (
    domain_stats[domain_stats["total_links"] >= 5]
    .sort_values("total_links", ascending=False)
    .head(10)
)
domain_stats_html = "<ul>"
for _, r in top_domains.iterrows():
    domain_stats_html += (
        f"<li><strong>{r['domain']}</strong>: "
        f"{r['broken_links']} / {r['total_links']} broken "
        f"({r['broken_pct']}%)</li>"
    )
domain_stats_html += "</ul>"


In [49]:
assert num_ok + num_dead + num_error  == num_total


In [50]:
from pathlib import Path
from datetime import date, datetime
from urllib.parse import quote
import pandas as pd


def save_sortable_html_df_regeringen(
    df,
    out_dir="resultsRegeringen",
    domains=None,
    issue_url="https://github.com/salgo60/SCB-Wikidata/issues/49",
):
    out_dir = Path(out_dir)
    out_dir.mkdir(exist_ok=True)

    today = date.today().strftime("%Y_%m_%d")
    out_path = out_dir / f"links_regeringen_v2_{today}.html"
    rerun_ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

     # --- Förbered data ---
    df = df.copy()  
    status_counts = df["status"].value_counts()

    num_ok = int(status_counts.get("ok", 0))
    num_dead = int(status_counts.get("dead", 0))
    num_error = int(status_counts.get("error", 0))
    num_total = len(df)
    
    pct_ok = round(100 * num_ok / num_total, 1)
    pct_dead = round(100 * num_dead / num_total, 1)
    pct_error = round(100 * num_error / num_total, 1)
    
    # Broken links: archived vs lost
    num_dead_archived = df[
        (df["status"] == "dead") & (df["ia_status"] == "available")
    ].shape[0]
    
    num_dead_lost = num_dead - num_dead_archived

    domains = domains or []
    from urllib.parse import urlparse
    
    df["domain"] = df["url"].apply(
        lambda u: urlparse(u).netloc.lower() if pd.notna(u) else None
    )

    domain_stats = (
        df
        .groupby("domain")
        .agg(
            total_links=("url", "count"),
            broken_links=("status", lambda s: (s == "dead").sum()),
            error_links=("status", lambda s: (s == "error").sum()),
        )
        .reset_index()
    )
    
    domain_stats["broken_pct"] = (
        100 * domain_stats["broken_links"] / domain_stats["total_links"]
    ).round(1)
    
    domain_stats["error_pct"] = (
        100 * domain_stats["error_links"] / domain_stats["total_links"]
    ).round(1)
    
    domain_stats["problem_pct"] = (
        100 * (domain_stats["broken_links"] + domain_stats["error_links"])
        / domain_stats["total_links"]
    ).round(1)
    

    domain_stats_html = "<ul>"
    for _, r in top_domains.iterrows():
        domain_stats_html += (
            f"<li><strong>{r['domain']}</strong>: "
            f"{r['broken_links']} / {r['total_links']} broken "
            f"({r['broken_pct']}%)</li>"
        )
    domain_stats_html += "</ul>"

    domain_table_html = (
        domain_stats
        .head(20)
        .to_html(
            classes="pivot",
            border=0,
            index=False,
        )
    )

    STATUS_ICON = {
        "ok":    ("fa-circle-check", "#2e7d32", "OK"),
        "dead":  ("fa-circle-xmark", "#c62828", "Broken link"),
        "error": ("fa-triangle-exclamation", "#ef6c00", "Request error"),
    }
    
    if "status" in df.columns:
        def render_status(r):
            icon, color, label = STATUS_ICON.get(
                r["status"], ("fa-question-circle", "#757575", "Unknown")
            )
            reason = r.get("reason", "")
            return (
                f'<span class="status-icon" '
                f'data-status="{r["status"]}" '
                f'title="{label}: {reason}" '
                f'style="color:{color}; font-size:14px; cursor:pointer;">'
                f'<i class="fa-solid {icon}"></i>'
                f'</span>'
            )
        df.insert(
            0,
            "status_icon",
            df.apply(render_status, axis=1)
        )

        def render_ia_icon(r):
            if r.get("ia_status") == "available" and r.get("ia_url"):
                return (
                    f'<a href="{r["ia_url"]}" target="_blank" '
                    f'title="Archived copy (Internet Archive)">'
                    f'<i class="fa-solid fa-box-archive" '
                    f'style="color:#1565c0;"></i>'
                    f'</a>'
                )
            return ""
        
        df.insert(
            1,
            "archive",
            df.apply(render_ia_icon, axis=1)
        )


    # Wikipedia: ikon + titel (byggd från lang + page_title)
    if {"lang", "page_title"}.issubset(df.columns):
        df["page_title"] = df.apply(
            lambda r: (
                f'<a href="https://{r["lang"]}.wikipedia.org/wiki/{quote(str(r["page_title"]))}" '
                f'target="_blank" title="Wikipedia ({r["lang"]})">'
                f'<i class="fa-brands fa-wikipedia-w" style="margin-right:6px;"></i>'
                f'{r["page_title"]}</a>'
                if pd.notna(r["lang"]) and pd.notna(r["page_title"])
                else r["page_title"]
            ),
            axis=1,
        )

    # Externa länkar
    for col in ["Wikipedia-länk", "Extern länk", "url"]:
        if col in df.columns:
            df[col] = df[col].apply(
                lambda x: f'<a href="{x}" target="_blank">{x}</a>' if pd.notna(x) else ""
            )

    # --- HTML-tabell ---
    html_table = df.to_html(
        classes="pivot",
        border=0,
        escape=False,  # krävs för HTML-länkar
        index=False,
    )

    # --- CSS ---
    css = """
    <style>
      body {
        font-family: Arial, sans-serif;
        margin: 20px;
      }
      table.pivot {
        border-collapse: collapse;
        width: 100%;
        font-size: 12px;
      }
      table.pivot th, table.pivot td {
        border: 1px solid #999;
        padding: 6px 8px;
        text-align: left;
        vertical-align: top;
        white-space: normal;
      }
      table.pivot th {
        cursor: pointer;
        background: #f2f2f2;
      }
      table.pivot th:hover {
        background: #e2e2e2;
      }
      table.pivot thead th {
        position: sticky;
        top: 0;
        background: #f2f2f2;
        z-index: 2;
      }
      table.pivot th::after {
        content: "";
        float: right;
        opacity: 0.4;
      }
      table.pivot th.sorted-asc::after {
        content: " ▲";
      }
      table.pivot th.sorted-desc::after {
        content: " ▼";
      }
      /* Row coloring by status */
      table.pivot tr[data-status="dead"] {
         background-color: #fdecea;  /* light red */
      }
      table.pivot tr[data-status="dead"] td:nth-child(2) i {
          color: #1565c0;
        }

      table.pivot tr[data-status="error"] {
          background-color: #fff4e5;  /* light orange */
      }

      table.pivot td a {
        color: #0645ad;
        text-decoration: none;
      }
      table.pivot td a:hover {
        text-decoration: underline;
      }
      .meta {
        background: #f8f8f8;
        border: 1px solid #ccc;
        padding: 12px;
        margin-bottom: 20px;
        font-size: 13px;
      }
      .meta h2 {
        margin-top: 0;
      }
    </style>
    """

    # --- JavaScript (sortering) ---
    js = """
    <script>
    document.addEventListener('DOMContentLoaded', () => {
        // Propagate status from first cell to row
        document.querySelectorAll("table.pivot tbody tr").forEach(row => {
            const statusCell = row.querySelector(".status-icon");
            if (statusCell) {
                row.dataset.status = statusCell.dataset.status;
            }
        });
        let showOnlyBroken = false;

        document.querySelectorAll(".status-icon").forEach(icon => {
            icon.addEventListener("click", event => {
                event.stopPropagation(); // prevent column sort
                showOnlyBroken = !showOnlyBroken;
        
                document.querySelectorAll("table.pivot tbody tr").forEach(row => {
                    if (showOnlyBroken) {
                        row.style.display =
                            row.dataset.status === "dead" ? "" : "none";
                    } else {
                        row.style.display = "";
                    }
                });
            });
        });

        document.querySelectorAll("table.pivot th").forEach((header, colIndex) => {
            header.addEventListener("click", () => {
                const table = header.closest("table");
                const tbody = table.querySelector("tbody");
                const rows = Array.from(tbody.querySelectorAll("tr"));
                const asc = !header.classList.contains("sorted-asc");

                rows.sort((a, b) => {
                    const A = a.children[colIndex].innerText.trim();
                    const B = b.children[colIndex].innerText.trim();
                    const numA = parseFloat(A.replace(",", "."));
                    const numB = parseFloat(B.replace(",", "."));
                    if (!isNaN(numA) && !isNaN(numB)) {
                        return asc ? numA - numB : numB - numA;
                    }
                    return asc ? A.localeCompare(B) : B.localeCompare(A);
                });

                table.querySelectorAll("th").forEach(th =>
                    th.classList.remove("sorted-asc", "sorted-desc")
                );
                header.classList.add(asc ? "sorted-asc" : "sorted-desc");
                rows.forEach(row => tbody.appendChild(row));
            });
        });
    });
    </script>
    """
    status_counts = df["status"].value_counts()
    
    num_ok = status_counts.get("ok", 0)
    num_dead = status_counts.get("dead", 0)
    num_error = status_counts.get("error", 0)
    num_total = len(df)

    # --- Metadata ---
    meta_html = f"""
    <div class="meta">
      <h2>Summary</h2>
    
      <p><strong>Rerun:</strong> {rerun_ts}</p>
      <p><strong>Script:</strong>
         <a href="{SCRIPT_URL}" target="_blank">{SCRIPT_NAME}</a>
      </p>
    
      <p>
        <strong>Links checked:</strong> {num_total}<br>
        <strong style="color:#2e7d32;">OK:</strong> {num_ok} ({pct_ok}%)<br>
        <strong style="color:#c62828;">Broken:</strong> {num_dead} ({pct_dead}%)<br>
        &nbsp;&nbsp;↳ Archived: {num_dead_archived}<br>
        &nbsp;&nbsp;↳ Lost: {num_dead_lost}<br>
        <strong style="color:#ef6c00;">Errors:</strong> {num_error} ({pct_error}%)
      </p>
      <p><strong>Issue:</strong>
         <a href="{issue_url}" target="_blank">{issue_url.split("/")[-1]}</a>
      </p>
    
      <p><strong>Datakällor:</strong><br>
         Wikidata<br>
         MediaWiki API – exturlusage
      </p>
    
      <h2>Domains with broken links</h2>
      <p>Top domains ranked by broken-link impact.</p>
      {domain_table_html}
    </div>
    """


    # --- Slutlig HTML ---
    html = f"""
    <html>
    <head>
      <meta charset="utf-8">
      <title>regeringen.se links in Wikipedia</title>
      <link rel="stylesheet"
            href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.1/css/all.min.css">
      {css}
    </head>
    <body>
      <h1>Wikipedia → regeringen v1</h1>
      {meta_html}
      <p>Sorterbar tabell. Klicka på kolumnrubriker för sortering.</p>
      {html_table}
      {js}
    </body>
    </html>
    """

    out_path.write_text(html, encoding="utf-8")
    print(f"✅ HTML skapad: {out_path}")


In [51]:

save_sortable_html_df_regeringen(df_regeringen)


✅ HTML skapad: resultsRegeringen/links_regeringen_v2_2026_01_09.html


In [52]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2026-01-09 05:14:58
Total time elapsed: 425 minutes 20.35 seconds
